In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn import preprocessing
import os
%matplotlib inline

In [ ]:


# train_dateを取得
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',parse_dates=["date"])
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

# １．全データの概要確認



In [ ]:
test.head()

In [ ]:
sales_train.head()

In [ ]:
shops.head()

In [ ]:
items.head()

In [ ]:
item_categories.head()

・ID - an Id that represents a (Shop, Item) tuple within the test set

・shop_id - unique identifier of a shop

・item_id - unique identifier of a product

・item_category_id - unique identifier of item category

・item_cnt_day - number of products sold. You are predicting a 
monthly amount of this measure

・item_price - current price of an item

・date - date in format dd/mm/yyyy

・date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

・item_name - name of item

・shop_name - name of shop

item_category_name - name of item category

# ２.big_categoryのカラムを作成
item_cagory_nameはPC - Гарнитуры/Наушники	のように
大分類-小分類という記法になっているので、大分類だけで再度まとめる

In [ ]:
# ' - 'で文字列分割
item_categories['big_category_name'] = item_categories['item_category_name'].map(lambda x: x.split(' - ')[0])
item_categories['big_category_name'].value_counts()

In [ ]:
# Чистые носители (штучные)とЧистые носители (шпиль)をЧистые носителиでまとめてしまう
item_categories.loc[
    item_categories['big_category_name']=='Чистые носители (штучные)','big_category_name'
] = 'Чистые носители'


item_categories.loc[
    item_categories['big_category_name']=='Чистые носители (шпиль)','big_category_name'
] = 'Чистые носители'

item_categories['big_category_name'].value_counts()

In [ ]:
item_categories

In [ ]:

# item_category_nameのカラムを消す
item_categories=item_categories.drop('item_category_name',axis=1)

In [ ]:
item_categories

In [ ]:
items

In [ ]:
# itemのDFとマージしてやる
items=items.merge(item_categories)

In [ ]:
items

**後々big_category_nameをlabel_encodingすることを忘れないように**


item_nameはあまり関係なさそうなのでdropする
item_category_idは後々big_categoryでlabel_encodingするので消す

In [ ]:
items=items.drop(['item_name','item_category_id'],axis=1)


In [ ]:
items

# ３.city_nameのカラムを作ってやる

shop_nameは、市名→店名みたいな構成らしい
ので、一番大きい分類である市の情報を使ってやる

shopの名前自体は関係ないだろう、よほどのブランドとかでない限り

In [ ]:

shops['city_name'] = shops['shop_name'].map(lambda x: x.split(' ')[0])

shops['city_name'].value_counts()

In [ ]:
# !ЯкутскとЯкутскを一緒にする
shops.loc[shops['city_name']=='!Якутск','city_name'] = 'Якутск'

shops['city_name'].value_counts()

In [ ]:
shops

In [ ]:
shops=shops.drop('shop_name',axis=1)
shops

# ４.月次shop_id*item_id別売上点数, 月次shop_id*item_id別売上金額のカラムを作ってやる

In [ ]:
# 日次売り上げ額
sales_train['date_sales'] = sales_train['item_cnt_day'] * sales_train['item_price']
sales_train

item_cnt_dayが負のやつはNaNとみなして消去する

In [ ]:
sales_train[sales_train['item_cnt_day']<0].count()

In [ ]:
sales_train[sales_train['item_cnt_day']<0]

In [ ]:
sales_train.index[sales_train['item_cnt_day']<0]

In [ ]:
sales_train=sales_train.drop(sales_train.index[sales_train['item_cnt_day']<0])
sales_train

In [ ]:
# 月次shop_id*item_id別売上点数のカラムを作る
mon_shop_item_cnt = sales_train[
    ['date_block_num','shop_id','item_id','item_cnt_day']
].groupby(
    ['date_block_num','shop_id','item_id'],
    as_index=False
).sum().rename(columns={'item_cnt_day':'mon_shop_item_cnt'})
mon_shop_item_cnt

In [ ]:
# shop_id、item_idのくくり関係なく月の合計売上点数をグラフ化して傾向を確認するためのグラフを作る
for_graph1=mon_shop_item_cnt[['date_block_num','mon_shop_item_cnt']].groupby('date_block_num').sum()
for_graph1

In [ ]:
# 月次shop_id*item_id別売上金額
mon_shop_item_sales = sales_train[
    ['date_block_num','shop_id','item_id','date_sales']
].groupby(
    ['date_block_num','shop_id','item_id'],
    as_index=False
).sum().rename(columns={'date_sales':'mon_shop_item_sales'})
mon_shop_item_sales

In [ ]:
# shop_id、item_idのくくり関係なく、月の合計売上額を表すグラフを作る
for_graph2=mon_shop_item_sales[['date_block_num','mon_shop_item_sales']].groupby('date_block_num').sum()
for_graph2

In [ ]:
# 月ごとの店、商品のくくりを除いた商品の合計売上額
plt.title('graph2')
plt.xlabel('date_block_num')
plt.ylabel('month_sales')
plt.plot('mon_shop_item_sales',data=for_graph2)

年販売額は周期的に変化していて毎年同じような値

12月が一番高くなる傾向
消費者の購買意欲が最も高くなる付きだといえる

テストデータは11月なのでほかの月より大きくなるはず

In [ ]:
# 月ごとの店、商品のくくりを除いた販売数の合計
plt.title('graph1')
plt.xlabel('date_block_num')
plt.ylabel('month_num')
plt.plot('mon_shop_item_cnt',data=for_graph1)

月販売数はジョジョに減少していってる
なんで？

販売額の高い商品ばかりが売れるようになった？


# ５.今まで作ったデータをまとめて可視化

In [ ]:
data_matome=mon_shop_item_cnt.merge(mon_shop_item_sales)
data_matome

In [ ]:
items

In [ ]:
shops.head()

In [ ]:
data_matome=data_matome.merge(shops)

In [ ]:
data_matome

In [ ]:
data_matome=data_matome.merge(items)

In [ ]:
data_matome

In [ ]:
for_graph3= data_matome.groupby(
    ['date_block_num','big_category_name'],
    as_index=False
).sum()
plt.figure(figsize=(20, 10))
sns.lineplot(x='date_block_num',y='mon_shop_item_cnt',data=for_graph3,hue='big_category_name')
plt.title('Montly item counts by big category')

In [ ]:
for_graph4= data_matome.groupby(
    ['date_block_num','city_name'],
    as_index=False
).sum()
plt.figure(figsize=(20, 10))
sns.lineplot(x='date_block_num',y='mon_shop_item_cnt',data=for_graph4,hue='city_name')
plt.title('Montly item counts by city')

# ６.trainデータを作成していく

In [ ]:
# テストデータセットに含まれる店舗ID×商品ID、かつ、月のシーケンス(date_block_num)が0~34の組み合わせを生成する


train_full_comb = pd.DataFrame()
for i in range(35):
    mid = test[['shop_id','item_id']]
    mid['date_block_num'] = i
    train_full_comb = pd.concat([train_full_comb,mid],axis=0)

In [ ]:
train_full_comb

In [ ]:
# 月次売上商品数をマージ
train_test = pd.merge(
    train_full_comb,
    mon_shop_item_cnt,
    on=['date_block_num','shop_id','item_id'],
    how='left'
)



In [ ]:
train_test

In [ ]:
# 月次売上金額をマージ
train_test = pd.merge(
    train_test,
    mon_shop_item_sales,
    on=['date_block_num','shop_id','item_id'],
    how='left'
)

In [ ]:
train_test

In [ ]:

# 学習データにマスタをマージ
# item_idのjoin
train_test = pd.merge(
    train_test,
    items[['item_id','big_category_name']],
    on='item_id',
    how='left'
)

In [ ]:
train_test

In [ ]:
items

In [ ]:
shops

In [ ]:
# shop_idのjoin
train_test = pd.merge(
    train_test,
    shops[['shop_id','city_name']],
    on='shop_id',
    how='left'
)

In [ ]:
train_test

True target values are clipped into [0,20] range.

という記述がKaggle側にあったので、mon_shop_item_cntの範囲を0から20に指定

In [ ]:
train_test['mon_shop_item_cnt'] = train_test['mon_shop_item_cnt'].clip(0,20)

# ７.ラグ特徴量作成


ラグは直近の1，2，3ヶ月前と一年ごとに周期性があったので、11，12か月まえのものも取ってやる

In [ ]:
# ラグ生成対象のカラム
lag_col_list = ['mon_shop_item_cnt','mon_shop_item_sales']
# ラグリスト(1,2,3,11,12か月前)
lag_num_list = [1,2,3,11,12]

# shop_id*item_id*date_block_numでソート
train_test = train_test.sort_values(
    ['shop_id', 'item_id','date_block_num'],
    ascending=[True, True,True]
).reset_index(drop=True)



In [ ]:
train_test

In [ ]:
# ラグ特徴量の生成
for lag_col in lag_col_list:
    for lag in lag_num_list:
        set_col_name =  lag_col + '_' +  str(lag)
        df_lag = train_test[['shop_id', 'item_id','date_block_num',lag_col]].sort_values(
            ['shop_id', 'item_id','date_block_num'],
            ascending=[True, True,True]
        ).reset_index(drop=True).shift(lag).rename(columns={lag_col: set_col_name})
        train_test = pd.concat([train_test, df_lag[set_col_name]], axis=1)

In [ ]:
train_test

In [ ]:
# NaNを０で埋めてやる、

# 疑問点、目的変数もおきかえてしまうことになるけど本当にそれでいいのか
train_test=train_test.fillna(0)

### 疑問点、目的変数もおきかえてしまうことになるけど本当にそれでいいのか

In [ ]:
train_test

# 8.trainデータとtestデータを分けてやる

#### ここで最大12か月前のデータを取得しているので、data_block_numが12以上のものを使ってやる

In [ ]:
train_=train_test[(train_test['date_block_num']<=33)&(train_test['date_block_num']>=12)].reset_index(drop=True)


In [ ]:
train_

In [ ]:
test_ = train_test[train_test['date_block_num']==34].reset_index(drop=True)
 

In [ ]:
test_

# 9.big_category_name,city_nameをlabel_encordingする

In [ ]:
from sklearn.preprocessing import LabelEncoder
obj_col_list = ['big_category_name','city_name']
for obj_col in obj_col_list:
    le = LabelEncoder()
    train_[obj_col] = pd.DataFrame({obj_col:le.fit_transform(train_[obj_col])})
    test_[obj_col] = pd.DataFrame({obj_col:le.fit_transform(test_[obj_col])})

# 10.特徴量と目的変数の分離

In [ ]:
train_y = train_['mon_shop_item_cnt']
train_X = train_.drop(columns=['mon_shop_item_cnt','mon_shop_item_sales','date_block_num'])
test_X = test_.drop(columns=['mon_shop_item_cnt','mon_shop_item_sales','date_block_num'])

In [ ]:
train_y

In [ ]:
train_X

In [ ]:
list(le.classes_)

# 11.model構築

In [ ]:
import xgboost as xgb
dm_train = xgb.DMatrix(train_X, label=train_y)

In [ ]:
param = {
    'max_depth': 8, 
    'eta': 0.1, 
    'objective': 'reg:squarederror',
    'colsample_bytree':1.0,
    'colsample_bylevel':0.3,
    'subsample':0.9,
    'gamma':0,
    'lambda':1,
    'alpha':0,
    'min_child_weight':1
    
}

In [ ]:
num_round=100,
model = xgb.train(param, dm_train)

In [ ]:
xgb.plot_importance(model)

In [ ]:
# 木の表示
xgb.to_graphviz(model)

In [ ]:
# テスト用のデータを生成
dm_test = xgb.DMatrix(test_X)

# 予測
y_pred = model.predict(dm_test)

In [ ]:
y_pred

In [ ]:
np.sqrt(
    np.mean(
        np.square(
            np.array(
                np.array(train_y) - model.predict(dm_train)
            )
        )
    )
)

In [ ]:
test_y = model.predict(dm_test)
test_X['item_cnt_month'] = test_y
submission = pd.merge(
    test,
    test_X[['shop_id','item_id','item_cnt_month']],
    on=['shop_id','item_id'],
    how='left'
)
# 提出ファイル作成
submission[['ID','item_cnt_month']].to_csv('submission1.csv', index=False)